In [3]:
#淘汰率计算
#叶中盛
#2022/07/12

In [1]:
import pandas
import math

In [2]:
data_source = pandas.read_csv('Vidas/export/Defect_RawData_0713.csv')
data_layer = pandas.read_csv('Vidas/export/Wafer_Summary_0713.csv')
data_merge = pandas.merge(data_source, data_layer, on='WAFER_LAYER_KEY')

C:\Users\luoyingying\AppData\Local\Temp\ipykernel_7728\2702848651.py:1: DtypeWarning: Columns (2,25,39) have mixed types. Specify dtype option on import or set low_memory=False.
  data_source = pandas.read_csv('Vidas/export/Defect_RawData_0713.csv')


In [8]:
# Migrate index x,y into gravity center based index x,y 2.
data_merge['center_bias_x'] = data_merge['CENTER_X']/data_merge['DIE_SIZE_X']
data_merge['center_bias_y'] = data_merge['CENTER_Y']/data_merge['DIE_SIZE_Y']
data_merge['index_bias_x'] = data_merge['center_bias_x'].apply(lambda x: int(x) if x>=0 else int(x)-1)
data_merge['index_bias_y'] = data_merge['center_bias_y'].apply(lambda x: int(x) if x>=0 else int(x)-1)
data_merge['index_int_x'] = data_merge['INDEX_X'] - data_merge['index_bias_x']
data_merge['index_int_y'] = data_merge['INDEX_Y'] - data_merge['index_bias_y']

In [12]:
#Only few rows change.
data_merge[data_merge['index_bias_x']>0][['center_bias_x','center_bias_y','index_bias_x','index_bias_y','index_x_2','index_y_2','INDEX_X','INDEX_Y']]

,center_bias_x,center_bias_y,index_bias_x,index_bias_y,index_x_2,index_y_2,INDEX_X,INDEX_Y
9583,1.146244,0.595333,1,0,15,18,16,18
9584,1.146244,0.595333,1,0,12,89,13,89
9585,1.146244,0.595333,1,0,11,-3,12,-3
9586,1.146244,0.595333,1,0,-5,-20,-4,-20
9587,1.146244,0.595333,1,0,-15,55,-14,55
...,...,...,...,...,...,...,...,...
628897,1.146244,0.595333,1,0,46,58,47,58
628898,1.146244,0.595333,1,0,43,-26,44,-26
628899,1.146244,0.595333,1,0,40,74,41,74
628900,1.146244,0.595333,1,0,40,11,41,11


In [44]:
#Layer, size, count requirements.
data_min = pandas.read_csv('defect_min.csv')
data_min = data_min[['USER_LAYER','USER_SIZE_OOC','USER_SIZE','PRODUCT']]
data_min.columns = ['LAYER_ID_x', 'USER_SIZE_OOC', 'USER_SIZE', 'PRODUCT_x']
data_y = data_merge[data_merge['PRODUCT_x']=='Y0145A1']
data_y = pandas.merge(data_y,data_min,on=['PRODUCT_x', 'LAYER_ID_x'])

#Select process required layer.
list_process = data_min['LAYER_ID_x'].tolist()
data_y['in_process'] = data_y['LAYER_ID_x'].apply(lambda x: x in list_process)
data_y = data_y[data_y['in_process']==1]

#Select minimum count required defect.
data_y_group = data_y.groupby('WAFER_LAYER_KEY').agg({'DEFECT_ID':'count'})
data_y_group.columns = ['defect_count']
data_y = pandas.merge(data_y,data_y_group,on=['WAFER_LAYER_KEY'])
data_y['count_limit'] = data_y['defect_count'] > data_y['USER_SIZE_OOC']
data_y = data_y[data_y['count_limit'] == 1]

#Select minimum size required defect.
data_y['defect_limit'] = data_y['SIZE_D'] > data_y['USER_SIZE']*1000
data_y = data_y[data_y['defect_limit'] == 1]



In [12]:
#Migrage golden map index x,y into gravity center based x,y.
die_array_center_x = 0
die_array_center_y = 0.44
die_width = 1.83
die_height = 1.57
index_bias_x = die_array_center_x/die_width
index_bias_y = die_array_center_y/die_height
print('index bias x:', index_bias_x)
print('index_bias_y:', index_bias_y)

index bias x: 0.0
index_bias_y: 0.2802547770700637


In [13]:
# Default effective area matrix
data_default = pandas.read_csv('Y0145A.csv',header=0)
data_default['list_default'] = data_default['default'].apply(lambda x:x.split(','))
list_matrix = [[ [] for i in range(-95,95)] for i in range(-82,82)]
int_total = 0
for i in data_default['list_default']:
    list_matrix[int(i[0])][int(i[1])] = int(i[2])
    if int(i[2]) == 0:
        int_total += 1
print('total green die count:', int_total)

total green die count: 22521


In [45]:
# Effective defect earlier than present layer.
list_layer = data_y['WAFER_LAYER_KEY'].unique()
dict_result = {}
for i in list_layer:
    # Dataframe earlier than present layer.
    str_time = data_y[data_y['WAFER_LAYER_KEY'] == i]['MEASURE_TIME_x'].unique()[0]
    str_lot = data_y[data_y['WAFER_LAYER_KEY'] == i]['LOT_x'].unique()[0]
    str_wafer = data_y[data_y['WAFER_LAYER_KEY'] == i]['WAFER_x'].unique()[0]
    data_temp = data_y[data_y['LOT_x'] == str_lot]
    data_temp = data_temp[data_temp['WAFER_x'] == str_wafer]
    data_temp = data_temp[data_temp['MEASURE_TIME_x'] <= str_time]
    int_defect = data_temp.shape[0]
    # Judge whether defect is effective.
    list_matrix_temp = [[ [] for i in range(-100,100)] for i in range(-100,100)]
    int_sum = 0
    for index,j in data_temp.iterrows():
        int_default = list_matrix[int(j['index_int_x'])][int(j['index_int_y'])]
        if int_default == 0:
            if list_matrix_temp[int(j['index_int_x'])][int(j['index_int_y'])] != 1:
                int_sum += 1
                list_matrix_temp[int(j['index_int_x'])][int(j['index_int_y'])] = 1
    dict_result[i] = [int_defect, int_sum]

In [15]:
dict_result

{}

total green die count: 22521


In [46]:
# Group by wafer layer key.
data_group = data_y.groupby('WAFER_LAYER_KEY').agg({'WAFER_LAYER_KEY': 'first', 'PRODUCT_x':'first','LOT_x':'first', 'WAFER_x':'first',  'LAYER_ID_x':'first','USER_SIZE_OOC':'first'})
data_group['defect_count'] = data_group['WAFER_LAYER_KEY'].apply(lambda x: dict_result[x][0])
data_group['defect_die_count'] = data_group['WAFER_LAYER_KEY'].apply(lambda x: dict_result[x][1])
#data_group = data_group[data_group['defect_count'] > data_group['USER_SIZE_OOC']]
data_group['defect_ratio'] = data_group['defect_die_count']/int_total


In [47]:
# Final result 2022/07/19
data_group.columns = ['WAFER_LAYER_KEY', 'PRODUCT', 'LOT', 'WAFER', 'LAYER_ID',
       'USER_SIZE_OOC', 'defect_count', 'defect_die_count', 'defect_ratio']

In [48]:
data_group

,WAFER_LAYER_KEY,PRODUCT,LOT,WAFER,LAYER_ID,USER_SIZE_OOC,defect_count,defect_die_count,defect_ratio
WAFER_LAYER_KEY,,,,,,,,,
1187932,1187932,Y0145A1,A1C1632,2,SG1_AEI,100,149,2,0.000089
1187963,1187963,Y0145A1,A1C1632,3,SG1_AEI,100,139,17,0.000755
1187994,1187994,Y0145A1,A1C1632,4,SG1_AEI,100,105,20,0.000888
1188029,1188029,Y0145A1,A1C1632,25,SG1_AEI,100,105,6,0.000266
1265598,1265598,Y0145A1,A170367,5,M1_AEI,1,203,4,0.000178
...,...,...,...,...,...,...,...,...,...
2108936,2108936,Y0145A1,A250058,6,SG2_AEI,100,6496,789,0.035034
2109018,2109018,Y0145A1,A250058,7,SG2_AEI,100,150,82,0.003641
2109078,2109078,Y0145A1,A250058,8,SG2_AEI,100,4057,357,0.015852


In [76]:
data_group = data_group[data_group['over_limit']==1]

In [49]:
data_group.to_csv('data_Y0145A_0727.csv')

In [7]:
# Example of SQL 

'''
import numpy as np
from scipy import signal
import xlrd
import pandas as pd t
import cv2
import operator
import math
import random

from matplotlib import pyplot as plt
import time
'''
import cx_Oracle #连接DB

import re  #正则表达式
import os #与文件夹相关
import datetime
#from PIL import Image
#import tensorflow as tf
#from tensorflow import keras
#from tensorflow.keras.preprocessing.image import ImageDataGenerator
#from keras.preprocessing import image
#from keras.models import load_model

#%%

SQL_GET_CP_WAFER_SUM_DATA = """
SELECT 
PRODUCT,     
LOT,       
WAFER,                
CP_TYPE,              
CP_TEST_PROG,             
to_char(MEAS_TIME,'YYYY-MM-DD HH:mm:ss') MEAS_TIME,  
to_char(MEAS_TIME,'YYYYMMDD-HHmmss')TIME,    
GOOD_DIE_COUNT,   
TOTAL_DIE_COUNT,  
ORIGINAL_WAFER_NOTCH,     
YIELD，                  
GROSS_DIE_COUNT,          
MAP_DATA,
WAFER_NO,
CORR_LOT,
SCRIBE_ID             
FROM CP_WAFER_SUM 
WHERE 1 = 1
AND meas_time >=to_date(:TIME_START,'yyyyMMddHH24MISS')
AND meas_time <=to_date(:TIME_END,'yyyyMMddHH24MISS')
and YIELD >= :YIELD_MIN
and YIELD <= :YIELD_MAX
and TOTAL_DIE_COUNT>=GROSS_DIE_COUNT 
"""

#%%
# recordsDB = {}
# # for i in range(len(OutPut["uint_key"])):
#     # recordsDB["product"] = None
#     # recordsDB["lot"] = None
#     # recordsDB["wafer"] = None
#     # recordsDB["cp_type"] = None
#     # recordsDB["cp_test_prog"] = None
#     # recordsDB["pattern_name"] = None
#     # recordsDB["reliability"] = None  
    
# recordsDB["product"] = 'Y0099A'
# recordsDB["lot"] = 'A220667.004'
# recordsDB["wafer"] = 'A220667.004#01'
# recordsDB["cp_type"] = 'Y0099A-TK-01'
# recordsDB["cp_test_prog"] = 'Y0099A-TK-01'
# recordsDB["pattern_name"] = 'CenterRadio'
# recordsDB["reliability"] = '0.9'
# recordsDB["update_time"] =  datetime.datetime.now()
# recordsDB["wafer_no"] = '03'
# recordsDB["corr_lot"] = 'A220667'
# recordsDB["scribe_id"] ='A220667_25'
# recordsDB["meas_time"] = '2022-04-13 08:04:47'
#%%
SQL_CP_MAP_SSA = """
insert into CP_MAP_SSA
(PRODUCT,LOT,WAFER, CP_TYPE,CP_TEST_PROG,PATTERN_NAME,RELIABILITY,UPDATE_TIME,WAFER_NO,CORR_LOT,SCRIBE_ID,MEAS_TIME)
VALUES
(
:PRODUCT, 
:LOT, 
:WAFER, 
:CP_TYPE,
:CP_TEST_PROG,
:PATTERN_NAME,
:RELIABILITY,
:UPDATE_TIME,
:WAFER_NO,
:CORR_LOT,
:SCRIBE_ID,
:MEAS_TIME
)
"""

#%%
SQL_GET_CP_BIN_INFO_DATA = """
select 
distinct PRODUCT,
BIN_NAME，
BIN_TYPE 
from CP_WAFER_BIN_SUM
"""

#%%
    #连接DB    
conn = cx_Oracle.connect('EDA/JkEdaTest#2022@192.168.6.149/jktedadb')#这里的顺序是用户名/密码@oracleserver的ip地址/数据库名字
cur = conn.cursor()

#data = sourcedb.execute(config_sql.SQL_GET_CP_WAFER_SUM_DATA,YIELD_MIN=config.yield_min,YIELD_MAX=config.yield_max,TIME_START=sLastDataTime,TIME_END=sMaxDataTime)
data = cur.execute(SQL_GET_CP_WAFER_SUM_DATA,YIELD_MIN=0,YIELD_MAX=100,TIME_START='20220413000000',TIME_END='20220414000000')




DatabaseError: DPI-1047: Cannot locate a 64-bit Oracle Client library: "The specified module could not be found". See https://cx-oracle.readthedocs.io/en/latest/user_guide/installation.html for help

In [8]:
import cx_Oracle #连接DB


#连接DB    
conn = cx_Oracle.connect('EDA/JkEdaTest#2022@192.168.6.149/jktedadb')#这里的顺序是用户名/密码@oracleserver的ip地址/数据库名字
cur = conn.cursor()

#data = sourcedb.execute(config_sql.SQL_GET_CP_WAFER_SUM_DATA,YIELD_MIN=config.yield_min,YIELD_MAX=config.yield_max,TIME_START=sLastDataTime,TIME_END=sMaxDataTime)
data = cur.execute(SQL_GET_CP_WAFER_SUM_DATA,YIELD_MIN=0,YIELD_MAX=100,TIME_START='20220413000000',TIME_END='20220414000000')


DatabaseError: DPI-1047: Cannot locate a 64-bit Oracle Client library: "The specified module could not be found". See https://cx-oracle.readthedocs.io/en/latest/user_guide/installation.html for help